In [2]:
import selenium

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import requests
import pandas as pd
import json
from dataclasses import dataclass, asdict
from datetime import datetime

# Database Setup and CRUD Functions

In [3]:
tdy = datetime.today()
date = f'Nov {tdy.day}, 2022'

wwl_db = {'cluster': "WWL", 
'uri': 'https://data.mongodb-api.com/app/data-qvnrx/endpoint/data/v1' ,
'db': "wwl" ,
'key': "5d9sGO28viSiX1HnJlOLN6QMqPqxYz6NIKVUMvEU8wXvAS0CPHMMHs2jF0UHKSCF"}



## Updating database

def submit_data(collection, docs, credential):
    headers = {'Content-Type': 'application/json', 'Access-Control-Request-Headers': '*','api-key': credential['key']}
    insert_url = f"{credential['uri']}/action/insertMany"
    Payload = json.dumps({"collection": collection, "database": credential['db'], "dataSource": credential['cluster'], "documents": docs})
    response = requests.request("POST", insert_url, headers=headers, data=Payload)
    return response

def update_many(collection, condition, doc,  credential):
    headers = {'Content-Type': 'application/json', 'Access-Control-Request-Headers': '*','api-key': credential['key']}
    updateMany_url = f"{credential['uri']}/action/updateMany"
    Payload = json.dumps({"collection": collection, "database": credential['db'], "dataSource": credential['cluster'], "filter": condition, "update":{"$set": doc}})
    response = requests.request("POST", updateMany_url, headers=headers, data=Payload)
    return response

def update_one(collection, condition, doc,  credential):
    headers = {'Content-Type': 'application/json', 'Access-Control-Request-Headers': '*','api-key': credential['key']}
    updateOne_url = f"{credential['uri']}/action/updateOne"
    Payload = json.dumps({"collection": collection, "database": credential['db'], "dataSource": credential['cluster'], "filter": condition, "update":{"$set": doc}})
    response = requests.request("POST", updateOne_url, headers=headers, data=Payload)
    return response

def delete_many(collection, condition, credential):
    headers = {'Content-Type': 'application/json', 'Access-Control-Request-Headers': '*','api-key': credential['key']}
    deleteOne_url = f"{credential['uri']}/action/deleteMany"
    Payload = json.dumps({"collection": collection, "database": credential['db'],"dataSource": credential['cluster'], "filter": condition})
    response = requests.request("POST", deleteOne_url, headers=headers, data=Payload)
    return response

## Get data from database

def get_data_json(collection, credential):
    headers = {'Content-Type': 'application/json', 'Access-Control-Request-Headers': '*','api-key': credential['key']}
    findAll_url =  f"{credential['uri']}/action/find"
    Payload = json.dumps({"collection": collection, "database":credential['db'], "dataSource": credential['cluster'], "filter": {}, "limit":5000})
    response = requests.request("POST", findAll_url, headers=headers, data=Payload)
    response_json = response.json()['documents']
    return response_json

def get_data_df(collection, credential):
    headers = {'Content-Type': 'application/json', 'Access-Control-Request-Headers': '*','api-key': credential['key']}
    findAll_url =  f"{credential['uri']}/action/find"
    Payload = json.dumps({"collection": collection, "database":credential['db'], "dataSource": credential['cluster'], "filter": {}, "limit":5000})
    response = requests.request("POST", findAll_url, headers=headers, data=Payload)
    response_json = response.json()['documents']
    df = pd.read_json(json.dumps(response_json))
    return df

# Data Schema Defination 

In [4]:
@dataclass
class Report:
    date: str
    access_point: str
    validation_type: str
    total: str


In [4]:
import requests
import pandas as pd
import json

wwl_db = {'cluster': "WWL", 
'uri': 'https://data.mongodb-api.com/app/data-qvnrx/endpoint/data/v1' ,
'db': "wwl" ,
'key': "5d9sGO28viSiX1HnJlOLN6QMqPqxYz6NIKVUMvEU8wXvAS0CPHMMHs2jF0UHKSCF"}


def get_data_df(collection, credential):
    headers = {'Content-Type': 'application/json', 'Access-Control-Request-Headers': '*','api-key': credential['key']}
    findAll_url =  f"{credential['uri']}/action/find"
    Payload = json.dumps({"collection": collection, "database":credential['db'], "dataSource": credential['cluster'], "filter": {}, "limit":5000})
    response = requests.request("POST", findAll_url, headers=headers, data=Payload)
    response_json = response.json()['documents']
    df = pd.read_json(json.dumps(response_json))
    df['date'] = df['date'].astype('str')
    df['date'] = [_.split(' ')[0] for _ in df['date'].values]
    return df

# get_data_df(collection='wwl_tickets', credential=wwl_db)

# Starting Browser & Loading page

E:\Users\abega\AppData\Local\Temp\ipykernel_17144\2586550305.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')


In [4]:
main_window = driver.current_window_handle
driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.CONTROL + Keys.TAB)
driver.switch_to.window(driver.window_handles[-1])

## CSS Selectors

In [5]:



reporting_tools_css = '#nova > div > div.min-h-screen.flex-none.pt-header.w-sidebar.bg-grad-sidebar.px-6.sidebar-hidden > h3.cursor-pointer.flex.items-center.font-normal.dim.text-white.mb-6.text-base.no-underline > span'
report_select_css ='#report'
option_css = '#report > option:nth-child(3)'
launch_css = '#button'
calnder_css = '#dateRange > div'

i = 1

col_1_css = f'#datatables63750325ca2081 > tbody > tr:nth-child(1) > td.sorting_1'
col_2_css = f'#datatables637479b5b8e6b1 > tbody > tr:nth-child({i}) > td:nth-child(2)'
col_3_css = f'#datatables637479b5b8e6b1 > tbody > tr:nth-child({i}) > td.text-right'
# //*[@id="datatables63750e01093c71"]
# #datatables63750e01093c71
# #datatables63750db6832a41


# Helper Functions

steps to collect data:
1. Login -- Uisng Login Function
2. Seleect report -- Using report selection Function
3. Switch tabs and collect table data -- Using Table data collection function
4. Process and submit scrapped dat to mongodb collection -- Using Data Entry Function


Utility Functions:
1. test_selector -- to test the existance of the css_selector for the web page element 

#### Utility function Defination

#### Procedural Funntions Defination

In [36]:
driver = webdriver.Chrome('./chromedriver.exe')
driver.get("https://eu.connectngo.com/login")
pw = 'LusailWW2022'
email = 'icerink@lww.com'

email_css = '#email'
pw_css ='#password'
sign_in_btn ='#app > main > div > div > div.d-flex.col-lg-4.align-items-center.bg-white.p-5 > div > div > form > div.d-flex.justify-content-between.align-items-center.m-0 > button'

def test_selector(selector, driver=driver):
    try:
        ele = driver.find_element(By.CSS_SELECTOR, selector)
        return ele
    except NoSuchElementException:
        return None

def login(user_name_selector, user_name, pw_selector, pw, btn_selector):
    un_ele = test_selector(user_name_selector)
    pw_ele = test_selector(pw_selector)
    btn_ele = test_selector(btn_selector)
    if (un_ele is not None) and (pw_ele is not None) and (btn_ele is not None):
        un_ele.clear()
        un_ele.send_keys(user_name)
        pw_ele.clear()
        pw_ele.send_keys(pw)
        btn_ele.click()
        return 'Logged In'
    else:
        return 'Failed to log in'

def select_report(i=3):
    reporting_tools_css = '#nova > div > div.min-h-screen.flex-none.pt-header.w-sidebar.bg-grad-sidebar.px-6.sidebar-hidden > h3.cursor-pointer.flex.items-center.font-normal.dim.text-white.mb-6.text-base.no-underline > span'
    option_css = f'#report > option:nth-child({i})'
    launch_css = '#button'
    reporting_tools = test_selector(reporting_tools_css)
    if reporting_tools is None:
        return None
    else:
        reporting_tools.click()
        time.sleep(3)
        report = test_selector(option_css)
        if report is None:
            return None
        else:
            report.click()
            launch = test_selector(launch_css)
            launch.click()

def get_table_data():
    date_css = '#dateRange > div > span'
    try:
        table = driver.find_element(By.TAG_NAME, 'table')
        rows = table.find_elements(By.TAG_NAME, 'tr')
    except NoSuchElementException:
        return None
    dt_rng = test_selector(date_css)
    table = []
    if dt_rng is None:
        return None
    else:
        for i in range(1, len(rows)):
            cols = rows[i].find_elements(By.TAG_NAME, 'td')
            table_rows = [dt_rng.text]
            for j in range(len(cols)):
                table_rows.append(cols[j].text)
            table.append(table_rows)
        return table

def scrape_aacess_point():

    data = get_table_data()
    if data is not None:
        db_data = []
        for rows in data:
            db_data.append(asdict(Report(date=rows[0], access_point=rows[1] , validation_type=rows[2] , total=rows[3])))
        delete_many(collection='wwl_tickets', condition={'date': db_data[0]['date']}, credential=wwl_db)
        submit_data(collection='wwl_tickets', docs=db_data, credential=wwl_db)


def scrapper_app():
    if login(user_name=email , user_name_selector=email_css , pw=pw , pw_selector=pw_css , btn_selector=sign_in_btn) == 'Logged In':
        select_report()
        print('here')
        time.sleep(3)
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(3)
        print('here 1')
        scrape_aacess_point()
        time.sleep(3)
        print('here')
        scrape_aacess_point()
        time.sleep(3)
        print('here 1')
        date_css = '#dateRange > div'
        date = test_selector(date_css)
        date.click()
        yesterday = 'body > div.daterangepicker.ltr.show-ranges.opensright > div.ranges > ul > li:nth-child(2)'
        yest = test_selector(yesterday)
        yest.click()
        submit_btn = '#main-container > form > div > button.btn.btn-primary'
        sub_btn = test_selector(submit_btn)
        sub_btn.click()
        time.sleep(3)
        scrape_aacess_point()
        print('here 2')

E:\Users\abega\AppData\Local\Temp\ipykernel_3248\2895124025.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')


In [ ]:
yest_css = 'body > div.daterangepicker.ltr.show-ranges.opensright > div.ranges > ul > li.active'
yest = test_selector(yest_css)
yest.click()
# time.sleep(3)
# data_01 = get_table_data()
# if data_01 is not None:
    # print('here 02')
    # delete_many(collection='wwl_tickets', condition={'date': data_01[0]['date']}, credential=wwl_db)
#     return data + data_01
# else:
#     return data 


In [37]:
scrapper_app()

here
here 1
here
here 1
here 2


In [14]:
select_report(i=3)

In [15]:
driver.switch_to.window(driver.window_handles[-1])

In [21]:

def get_table_data():
    date_css = '#dateRange > div > span'
    try:
        table = driver.find_element(By.TAG_NAME, 'table')
        rows = table.find_elements(By.TAG_NAME, 'tr')
    except NoSuchElementException:
        return None
    dt_rng = test_selector(date_css)
    table = []
    if dt_rng is None:
        return None
    else:
        for i in range(1, len(rows)):
            cols = rows[i].find_elements(By.TAG_NAME, 'td')
            table_rows = [dt_rng.text]
            for j in range(len(cols)):
                table_rows.append(cols[j].text)
            table.append(table_rows)
        return table


In [22]:
get_table_data()

[['11-21-2022 - 11-21-2022', 'Cocomelon', 'Ok', '92'],
 ['11-21-2022 - 11-21-2022',
  'Cocomelon',
  'No access through access point',
  '10'],
 ['11-21-2022 - 11-21-2022', 'Exit', 'Wallet has no valid ticket', '716'],
 ['11-21-2022 - 11-21-2022', 'Exit', 'Event out of date', '6'],
 ['11-21-2022 - 11-21-2022', 'Exit', 'Ticket out of date', '39'],
 ['11-21-2022 - 11-21-2022', 'Exit', 'No access through access point', '31'],
 ['11-21-2022 - 11-21-2022',
  'Ice Rink',
  'No access through access point',
  '1'],
 ['11-21-2022 - 11-21-2022', 'Main Entrance', 'Ok', '2,336'],
 ['11-21-2022 - 11-21-2022', 'Main Entrance', 'Uid not found', '8'],
 ['11-21-2022 - 11-21-2022', 'Main Entrance', 'Uid not valid', '4'],
 ['11-21-2022 - 11-21-2022',
  'Main Entrance',
  'Wallet has no valid ticket',
  '20'],
 ['11-21-2022 - 11-21-2022', 'Main Entrance', 'Event out of date', '4'],
 ['11-21-2022 - 11-21-2022', 'Main Entrance', 'Ticket out of date', '3'],
 ['11-21-2022 - 11-21-2022', 'Main Entrance', 'Alr

In [16]:
delete_data(collection='wwl_tickets', condition={'date': '11-19-2022 - 11-19-2022'}, credential=wwl_db)

<Response [200]>

In [47]:
scrapper_app()

In [24]:
submit_data(collection='wwl_tickets', docs=data, credential=wwl_db)

<Response [201]>

In [10]:
driver.switch_to.window(driver.window_handles[-1])

In [82]:
get_data_df(collection='wwl_tickets', credential=wwl_db)

,_id,date,access_point,validation_type,total
0,6375117ecef5839887893a74,2022-11-02 11:00:00-20:22,Ice Rink,Uid not found,2
1,6375117ecef5839887893a75,2022-11-02 11:00:00-20:22,Ice Rink,Event out of date,13
2,6375117ecef5839887893a76,2022-11-02 11:00:00-20:22,Ice slide,Uid not found,2
3,6375117ecef5839887893a77,2022-11-02 11:00:00-20:22,Ice slide,Event out of date,13
4,6375117ecef5839887893a78,2022-11-02 11:00:00-20:22,,,30
...,...,...,...,...,...
85,637512998f59cc2335149f84,2022-11-16 11:00:00-20:22,Main Entrance,Uid not valid,14
86,637512998f59cc2335149f85,2022-11-16 11:00:00-20:22,Main Entrance,Wallet has no valid ticket,9
87,637512998f59cc2335149f86,2022-11-16 11:00:00-20:22,Main Entrance,Ticket out of date,5
88,637512998f59cc2335149f87,2022-11-16 11:00:00-20:22,Main Entrance,Already in zone,99


'Circus'

In [11]:
data = get_table_data()
data[:2]

TypeError: 'NoneType' object is not subscriptable

In [81]:
submit_data(collection='wwl_tickets', docs=data, credential=wwl_db)

<Response [201]>

In [ ]:



def get_table_data():
    
    col_1_css = f'#datatables637479b5b8e6b1 > tbody > tr:nth-child({i}) > td.sorting_1'
    col_2_css = f'#datatables637479b5b8e6b1 > tbody > tr:nth-child({i}) > td:nth-child(2)'
    col_3_css = f'#datatables637479b5b8e6b1 > tbody > tr:nth-child({i}) > td.text-right'
    test_selector(col_1_css)
    col_1_ele = driver.find_element(By.CSS_SELECTOR, col_1_css)
    col_2_ele = driver.find_element(By.CSS_SELECTOR, col_2_css)
    col_3_ele = driver.find_element(By.CSS_SELECTOR, col_3_css)

In [7]:
# ele = driver.find_element(By.CSS_SELECTOR, option_css)
btn_ele = driver.find_element(By.CSS_SELECTOR, calnder_css)

In [8]:
btn_ele.text

'Validation result by access point'